In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [4]:
data= pd.read_excel(r'/content/Telecom Churn Rate Dataset.xlsx')
data.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
data.drop('customerID',axis=1,inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True)
data.drop('index',axis=1,inplace=True)

In [ ]:
def column_info(data):
    cols=[]
    dtype=[]
    unique=[]
    nunique=[]
    null=[]

      for col in data.columns:
        cols.append(col)
        dtype.append(data[col].dtype)
        unique.append(data[col].unique())
        nunique.append(data[col].nunique())
        null.append(data[col].isna().sum()/len(data))

return pd.DataFrame({'cols':cols,'data type':dtype,'unique':unique,'nunqiue':nunique,'nulls':null})

In [ ]:
column_info(data)

In [ ]:
data['TotalCharges'].unique()

In [ ]:
data['TotalCharges'].value_counts()

In [ ]:
data['TotalCharges']=pd.to_numeric(data['TotalCharges'],errors='coerce')

In [ ]:
data['TotalCharges'].isna().sum()

In [ ]:
data['TotalCharges'].fillna(data['TotalCharges'].median(),inplace=True)

In [ ]:
data['MonthlyCharges'].value_counts()

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(data['MonthlyCharges'])
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(data['TotalCharges'])
plt.show()

In [ ]:
def check_outliers(data):
    q1=data['TotalCharges'].quantile(0.25)
    q3=data['TotalCharges'].quantile(0.75)
    iqr=q3-q1

    lower= q1-1.5*iqr
    upper=q3+1.5*iqr

    outliers=[]
    for i in range(len(data)):
        if data.loc[i,'TotalCharges'] > upper or data.loc[i,'TotalCharges'] < lower:
            outliers.append(data.loc[i,'TotalCharges'])
    print('outliers')

In [ ]:
check_outliers(data)

In [ ]:
def check_outlier(data):
    q1=data['MonthlyCharges'].quantile(0.25)
    q3=data['MonthlyCharges'].quantile(0.75)
    iqr=q3-q1

    lower= q1-1.5*iqr
    upper=q3+1.5*iqr

    outliers=[]
    for i in range(len(data)):
        if data.loc[i,'MonthlyCharges'] > upper or data.loc[i,'MonthlyCharges'] < lower:
            outliers.append(data.loc[i,'MonthlyCharges'])
    print('outliers')

In [ ]:
check_outlier(data)

In [ ]:
column_info(data)

In [ ]:
plt.figure(figsize=(50,25))
sns.heatmap(data.corr(),annot=True)
plt.show()

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
data['Churn']=label_encoder.fit_transform(data['Churn'])
data['Churn'].unique()

In [ ]:
data=pd.get_dummies(data)
data.head()

In [ ]:
data.info()

In [ ]:
data['Churn']

In [ ]:
data.corr()['Churn'][:-1]

In [ ]:
plt.figure(figsize=(50,25))
sns.heatmap(data.corr(),annot=True)
plt.show()

In [ ]:
x=data.drop(['Churn'],axis=1).values
y=data['Churn'].values

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2)

In [ ]:
from imblearn.combine import SMOTETomek
st=SMOTETomek()
n_train_x,n_train_y=st.fit_resample(train_x,train_y)

In [ ]:
## rondom forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(n_train_x,n_train_y)

In [ ]:
from sklearn.metrics import accuracy_score,recall_score ,precision_score, f1_score, confusion_matrix
def evalute(test_x,test_y,model):
    y_pred=model.predict(test_x)
    acc=accuracy_score(test_y,y_pred)
    recall=recall_score(test_y,y_pred)
    precision=precision_score(test_y,y_pred)
    f1=f1_score(test_y,y_pred)
    cm=confusion_matrix(test_y,y_pred)
    specificity=cm[0,0]/(cm[0,1]+cm[0,0])

return pd.Series({'accuracy_score':acc,'recall_score':recall,'precision_score':precision,'f1_score':f1,'specificity':specificity})


In [ ]:
evalute(train_x,train_y,model)

In [ ]:
evalute(test_x,test_y,model) # overfiting

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[50,100,200,300,500]
max_depth=[5,6,7,8,9,10]
min_samples_split=[3,4,5,6]

random_grid={'n_estimators':n_estimators,
             'max_depth':max_depth,
             'min_samples_split':min_samples_split}

In [ ]:
rs=RandomizedSearchCV(RandomForestClassifier(),param_distributions=random_grid,scoring='accuracy',cv=10,n_jobs=-1)
rs.fit(n_train_x,n_train_y)

In [ ]:
rs.best_params_

In [ ]:

model=RandomForestClassifier(**rs.best_params_)
model.fit(n_train_x,n_train_y)

In [ ]:
evalute(train_x,train_y,model)

In [ ]:
evalute(test_x,test_y,model) 

In [ ]:
plt.figure(figsize=(20,10))
plt.barh(data.columns[:-1],model.feature_importances_)
plt.show()